# 📓 The GenAI Revolution Cookbook

**Title:** Flow Blockchain: How to Build a Crypto App with AI Tools from Scratch

**Description:** Ship Flow Blockchain crypto app faster using AI-assisted Cadence, FCL integration, and testnet deployment, with security checks and troubleshooting steps.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



\#\# Why This Approach Works

Flow is purpose\-built for consumer\-scale blockchain applications with human\-readable smart contracts (Cadence), resource\-oriented safety, and low transaction costs. For AI Builders, Flow offers a clean environment to prototype token logic, wallet interactions, and AI\-guided UX without wrestling with gas optimization or cryptic error messages.

Integrating AI into the development loop transforms how users interact with blockchain transactions. Instead of raw error codes or silent failures, your dApp can explain what's happening in plain language, suggest fixes, and guide users through setup steps. This makes your application accessible to non\-technical users and accelerates debugging during development.

This tutorial walks you through building a single, focused project: a FungibleToken\-compatible token contract on Flow with a React frontend that uses AI to explain transactions and errors in real time. You'll deploy to testnet, connect a wallet, and validate the full flow end\-to\-end.

\#\# How It Works

The system has three main components:

\*\*Smart Contract Layer\*\*: A Cadence contract implementing the FungibleToken standard with Vault resources, minting capabilities, and event emission. This contract is deployed to Flow testnet and manages token state on\-chain.

\*\*Frontend Layer\*\*: A React app using FCL (Flow Client Library) for wallet authentication, transaction signing, and script execution. Users connect their wallet, set up their vault, and transfer tokens through a simple UI.

\*\*AI Explanation Layer\*\*: A Node.js Express server that calls OpenAI's API to generate plain\-language explanations of transactions and errors. The frontend sends transaction context (action, amount, recipient, error) to this server and displays the AI response to the user.

\*\*Data Flow\*\*: User initiates action → Frontend calls AI server for explanation → User reviews and confirms → Frontend sends transaction via FCL → Flow network executes → Frontend polls transaction status → AI server explains result or error.

\#\# Setup \& Installation

Install the Flow CLI:

\`\`\`bash
sh \-ci "$(curl \-fsSL https://raw.githubusercontent.com/onflow/flow\-cli/master/install.sh)"
\`\`\`

Verify installation:

\`\`\`bash
flow version
\`\`\`

Create a new Flow project:

\`\`\`bash
mkdir buildertoken\-dapp \&\& cd buildertoken\-dapp
flow init
\`\`\`

This generates a \`flow.json\` configuration file for managing accounts, contracts, and networks.

Install Node.js dependencies for the AI server:

\`\`\`bash
mkdir ai\-server \&\& cd ai\-server
npm init \-y
npm install express cors dotenv openai
cd ..
\`\`\`

Create a React frontend with Vite:

\`\`\`bash
npm create vite@latest frontend \-\- \-\-template react\-ts
cd frontend
npm install
npm install @onflow/fcl
cd ..
\`\`\`

Set up environment variables. Create \`ai\-server/.env\`:

\`\`\`
AI\_API\_KEY\=your\-openai\-api\-key
\`\`\`

Create \`frontend/.env\`:

\`\`\`
VITE\_BUILDER\_TOKEN\_ADDRESS\=0xYourContractAddress
\`\`\`

You'll update \`VITE\_BUILDER\_TOKEN\_ADDRESS\` after deploying the contract.

\#\# Step\-by\-Step Implementation

\#\#\# Write the Cadence Contract

Create \`cadence/contracts/BuilderToken.cdc\`. This contract implements the FungibleToken standard with safe mint/burn, events, and capability management:

\`\`\`cadence
import FungibleToken from 0x9a0766d93b6608b7

pub contract BuilderToken: FungibleToken {

 pub let VaultStoragePath: StoragePath
 pub let ReceiverPublicPath: PublicPath
 pub let BalancePublicPath: PublicPath
 pub let MinterStoragePath: StoragePath

 pub var totalSupply: UFix64

 pub event TokensInitialized(initialSupply: UFix64\)
 pub event TokensWithdrawn(amount: UFix64, from: Address?)
 pub event TokensDeposited(amount: UFix64, to: Address?)
 pub event TokensMinted(amount: UFix64, to: Address?)
 pub event TokensBurned(amount: UFix64, from: Address?)

 pub resource Vault: FungibleToken.Provider, FungibleToken.Receiver, FungibleToken.Balance {

 pub var balance: UFix64

 init(balance: UFix64\) {
 self.balance \= balance
 }

 pub fun withdraw(amount: UFix64\): @FungibleToken.Vault {
 pre {
 self.balance \>\= amount: "Insufficient balance"
 }
 self.balance \= self.balance \- amount
 emit TokensWithdrawn(amount: amount, from: self.owner?.address)
 return \<\-create Vault(balance: amount)
 }

 pub fun deposit(from: @FungibleToken.Vault) {
 let vault \<\- from as! @BuilderToken.Vault
 self.balance \= self.balance \+ vault.balance
 emit TokensDeposited(amount: vault.balance, to: self.owner?.address)
 vault.balance \= 0\.0
 destroy vault
 }

 destroy() {
 BuilderToken.totalSupply \= BuilderToken.totalSupply \- self.balance
 }
 }

 pub fun createEmptyVault(): @Vault {
 return \<\-create Vault(balance: 0\.0\)
 }

 pub resource Minter {

 pub fun mintTokens(amount: UFix64\): @BuilderToken.Vault {
 pre {
 amount \> 0\.0: "Amount must be positive"
 }
 BuilderToken.totalSupply \= BuilderToken.totalSupply \+ amount
 emit TokensMinted(amount: amount, to: nil)
 return \<\-create Vault(balance: amount)
 }
 }

 init() {
 self.totalSupply \= 0\.0

 self.VaultStoragePath \= /storage/BuilderTokenVault
 self.ReceiverPublicPath \= /public/BuilderTokenReceiver
 self.BalancePublicPath \= /public/BuilderTokenBalance
 self.MinterStoragePath \= /storage/BuilderTokenMinter

 let vault \<\- create Vault(balance: self.totalSupply)
 self.account.save(\<\-vault, to: self.VaultStoragePath)

 self.account.link\<\&BuilderToken.Vault{FungibleToken.Receiver}\>(
 self.ReceiverPublicPath,
 target: self.VaultStoragePath
 )

 self.account.link\<\&BuilderToken.Vault{FungibleToken.Balance}\>(
 self.BalancePublicPath,
 target: self.VaultStoragePath
 )

 let minter \<\- create Minter()
 self.account.save(\<\-minter, to: self.MinterStoragePath)

 emit TokensInitialized(initialSupply: self.totalSupply)
 }
}
\`\`\`

\#\#\# Write Cadence Scripts

Create \`cadence/scripts/get\_balance.cdc\` to query a user's token balance:

\`\`\`cadence
import BuilderToken from 0xBuilderToken
import FungibleToken from 0x9a0766d93b6608b7

pub fun main(account: Address): UFix64 {
 let cap \= getAccount(account)
 .getCapability\<\&BuilderToken.Vault{FungibleToken.Balance}\>(BuilderToken.BalancePublicPath)
 let ref \= cap.borrow() ?? panic("No Balance capability")
 return ref.balance
}
\`\`\`

\#\#\# Write Cadence Transactions

Create \`cadence/transactions/setup\_account.cdc\` to initialize a user's vault:

\`\`\`cadence
import BuilderToken from 0xBuilderToken
import FungibleToken from 0x9a0766d93b6608b7

transaction {
 prepare(acct: AuthAccount) {
 if acct.borrow\<\&BuilderToken.Vault\>(from: BuilderToken.VaultStoragePath) \=\= nil {
 acct.save(\<\-BuilderToken.createEmptyVault(), to: BuilderToken.VaultStoragePath)
 acct.link\<\&BuilderToken.Vault{FungibleToken.Receiver}\>(
 BuilderToken.ReceiverPublicPath,
 target: BuilderToken.VaultStoragePath
 )
 acct.link\<\&BuilderToken.Vault{FungibleToken.Balance}\>(
 BuilderToken.BalancePublicPath,
 target: BuilderToken.VaultStoragePath
 )
 }
 }
}
\`\`\`

Create \`cadence/transactions/mint\_tokens.cdc\` for minting (admin only):

\`\`\`cadence
import BuilderToken from 0xBuilderToken
import FungibleToken from 0x9a0766d93b6608b7

transaction(recipient: Address, amount: UFix64\) {
 prepare(signer: AuthAccount) {
 let minterRef \= signer.borrow\<\&BuilderToken.Minter\>(from: BuilderToken.MinterStoragePath)
 ?? panic("Could not borrow minter reference")

 let receiverRef \= getAccount(recipient)
 .getCapability(BuilderToken.ReceiverPublicPath)
 .borrow\<\&{FungibleToken.Receiver}\>()
 ?? panic("Could not borrow receiver reference")

 let mintedVault \<\- minterRef.mintTokens(amount: amount)
 receiverRef.deposit(from: \<\-mintedVault)
 }
}
\`\`\`

Create \`cadence/transactions/transfer\_tokens.cdc\` for peer\-to\-peer transfers:

\`\`\`cadence
import BuilderToken from 0xBuilderToken
import FungibleToken from 0x9a0766d93b6608b7

transaction(recipient: Address, amount: UFix64\) {
 prepare(signer: AuthAccount) {
 let vaultRef \= signer.borrow\<\&BuilderToken.Vault\>(from: BuilderToken.VaultStoragePath)
 ?? panic("Could not borrow vault reference")

 let receiverRef \= getAccount(recipient)
 .getCapability(BuilderToken.ReceiverPublicPath)
 .borrow\<\&{FungibleToken.Receiver}\>()
 ?? panic("Could not borrow receiver reference")

 let sentVault \<\- vaultRef.withdraw(amount: amount)
 receiverRef.deposit(from: \<\-sentVault)
 }
}
\`\`\`

\#\#\# Deploy to Testnet

Update \`flow.json\` to include your testnet account. Generate a new testnet account at \[Flow Testnet Faucet](https://testnet\-faucet.onflow.org/), then add it to \`flow.json\`:

\`\`\`json
{
 "accounts": {
 "testnet\-account": {
 "address": "0xYourTestnetAddress",
 "key": "your\-private\-key\-hex"
 }
 },
 "contracts": {
 "BuilderToken": "./cadence/contracts/BuilderToken.cdc"
 },
 "deployments": {
 "testnet": {
 "testnet\-account": \["BuilderToken"]
 }
 }
}
\`\`\`

Deploy the contract:

\`\`\`bash
flow project deploy \-\-network\=testnet
\`\`\`

After deployment, update \`frontend/.env\` with the deployed contract address.

\#\#\# Build the AI Explanation Server

Create \`ai\-server/index.js\`. This Express server receives transaction context and returns AI\-generated explanations:

\`\`\`js
import express from "express";
import cors from "cors";
import { config } from "dotenv";
import OpenAI from "openai";

config();

function ensureEnvVars(requiredKeys) {
 const missing \= requiredKeys.filter((k) \=\> !process.env\[k]);
 if (missing.length \> 0\) {
 throw new Error(
 \`Missing required environment variables: ${missing.join(", ")}\\n\` \+
 "Please set them in your .env file or environment."
 );
 }
}

ensureEnvVars(\["AI\_API\_KEY"]);

const app \= express();
app.use(cors());
app.use(express.json());

const client \= new OpenAI({ apiKey: process.env.AI\_API\_KEY });

app.post("/explain", async (req, res) \=\> {
 const { action, amount, recipient, error } \= req.body;

 const prompt \= \`
User action: ${action}
Amount: ${amount}
Recipient: ${recipient}
Error: ${error \|\| "none"}
Explain in plain language what this transaction will do on Flow with Cadence constraints.
If there's an error, propose concrete fixes.
\`;

 try {
 const r \= await client.chat.completions.create({
 model: "gpt\-4o\-mini",
 messages: \[{ role: "user", content: prompt }],
 temperature: 0\.2
 });
 res.json({ explanation: r.choices\[0].message.content });
 } catch (err) {
 console.error("AI explanation error:", err);
 res.status(500\).json({ error: "Failed to get explanation from AI." });
 }
});

app.listen(8787, () \=\> console.log("AI server running on http://localhost:8787"));
\`\`\`

Update \`ai\-server/package.json\` to enable ES modules:

\`\`\`json
{
 "type": "module",
 "scripts": {
 "start": "node index.js"
 }
}
\`\`\`

Start the AI server:

\`\`\`bash
cd ai\-server
npm start
\`\`\`

\#\#\# Build the React Frontend

Configure FCL in \`frontend/src/fclConfig.ts\`:

\`\`\`ts
import \* as fcl from "@onflow/fcl";

export function configureFCL() {
 fcl.config()
 .put("app.detail.title", "BuilderToken dApp")
 .put("accessNode.api", "https://rest\-testnet.onflow.org")
 .put("discovery.wallet", "https://fcl\-discovery.onflow.org/testnet/authn")
 .put("0xBuilderToken", import.meta.env.VITE\_BUILDER\_TOKEN\_ADDRESS)
 .put("0xFungibleToken", "0x9a0766d93b6608b7");
}

export default fcl;
\`\`\`

Create wallet authentication in \`frontend/src/App.tsx\`:

\`\`\`tsx
import { useEffect, useState } from "react";
import \* as fcl from "@onflow/fcl";
import { configureFCL } from "./fclConfig";
import Transfer from "./components/Transfer";

configureFCL();

type User \= { addr?: string \| null };

export default function App() {
 const \[user, setUser] \= useState({});

 useEffect(() \=\> {
 const unsubscribe \= fcl.currentUser().subscribe(setUser);
 return () \=\> unsubscribe();
 }, \[]);

 return (

# BuilderToken dApp

{!user.addr ? ( fcl.authenticate()}\>Connect Wallet

) : (
 \<\> Connected: {user.addr}fcl.unauthenticate()}\>Disconnect

)} );
}
\`\`\`

Create transaction logic in \`frontend/src/flow/transactions.ts\`:

\`\`\`ts
import \* as fcl from "@onflow/fcl";

export const SETUP \= \`
import BuilderToken from 0xBuilderToken
import FungibleToken from 0xFungibleToken
transaction {
 prepare(acct: AuthAccount) {
 if acct.borrow\<\&BuilderToken.Vault\>(from: BuilderToken.VaultStoragePath) \=\= nil {
 acct.save(\<\-BuilderToken.createEmptyVault(), to: BuilderToken.VaultStoragePath)
 acct.link\<\&BuilderToken.Vault{FungibleToken.Receiver}\>(
 BuilderToken.ReceiverPublicPath,
 target: BuilderToken.VaultStoragePath
 )
 acct.link\<\&BuilderToken.Vault{FungibleToken.Balance}\>(
 BuilderToken.BalancePublicPath,
 target: BuilderToken.VaultStoragePath
 )
 }
 }
}
\`;

export const TRANSFER \= \`
import BuilderToken from 0xBuilderToken
import FungibleToken from 0xFungibleToken
transaction(recipient: Address, amount: UFix64\) {
 prepare(signer: AuthAccount) {
 let vaultRef \= signer.borrow\<\&BuilderToken.Vault\>(from: BuilderToken.VaultStoragePath)
 ?? panic("Could not borrow vault reference")
 let receiverRef \= getAccount(recipient)
 .getCapability(BuilderToken.ReceiverPublicPath)
 .borrow\<\&{FungibleToken.Receiver}\>()
 ?? panic("Could not borrow receiver reference")
 let sentVault \<\- vaultRef.withdraw(amount: amount)
 receiverRef.deposit(from: \<\-sentVault)
 }
}
\`;

export async function sendTx(cadence: string, args: any\[]): Promise {
 const txId \= await fcl.mutate({
 cadence,
 args: (arg, t) \=\> args.map((\[val, type]) \=\> arg(val, type)),
 limit: 9999
 });
 return fcl.tx(txId).onceSealed();
}
\`\`\`

Create the Transfer component in \`frontend/src/components/Transfer.tsx\`:

\`\`\`tsx
import { useState } from "react";
import \* as fcl from "@onflow/fcl";
import { SETUP, TRANSFER } from "../flow/transactions";

export default function Transfer() {
 const \[recipient, setRecipient] \= useState("");
 const \[amount, setAmount] \= useState("10\.0");
 const \[explanation, setExplanation] \= useState("");
 const \[status, setStatus] \= useState("");

 async function explain(action: string, err?: string) {
 try {
 const r \= await fetch("http://localhost:8787/explain", {
 method: "POST",
 headers: { "Content\-Type": "application/json" },
 body: JSON.stringify({ action, amount, recipient, error: err })
 });
 const data \= await r.json();
 setExplanation(data.explanation);
 } catch (e) {
 setExplanation("Failed to get AI explanation.");
 }
 }

 async function setup() {
 setStatus("Setting up vault...");
 try {
 const txId \= await fcl.mutate({ cadence: SETUP, limit: 9999 });
 await fcl.tx(txId).onceSealed();
 setStatus("Vault setup complete");
 await explain("setup\_account");
 } catch (e: any) {
 setStatus("Setup failed");
 await explain("setup\_account", e.message);
 }
 }

 async function transfer() {
 setStatus("Submitting transfer...");
 try {
 await explain("transfer\_preview");
 const txId \= await fcl.mutate({
 cadence: TRANSFER,
 args: (arg, t) \=\> \[arg(recipient, t.Address), arg(amount, t.UFix64\)],
 limit: 9999
 });
 const sealed \= await fcl.tx(txId).onceSealed();
 setStatus(\`Transfer complete: ${sealed.statusString}\`);
 await explain("transfer\_confirm");
 } catch (e: any) {
 setStatus("Transfer failed");
 await explain("transfer\_error", e.message);
 }
 }

 return (

### Transfer BuilderToken

 setRecipient(e.target.value)}
 style\={{ marginRight: "0\.5rem", padding: "0\.5rem" }}
 /\>  setAmount(e.target.value)}
 style\={{ marginRight: "0\.5rem", padding: "0\.5rem" }}
 /\> Setup Vault

Send

{status}

In [ ]:
{explanation}

);
}
\`\`\`

Start the frontend:

\`\`\`bash
cd frontend
npm run dev
\`\`\`

\#\# Run and Validate

Start the AI server in one terminal:

\`\`\`bash
cd ai\-server
npm start
\`\`\`

Start the React frontend in another terminal:

\`\`\`bash
cd frontend
npm run dev
\`\`\`

Open your browser to \`http://localhost:5173\`. Click "Connect Wallet" and authenticate with a Flow testnet wallet (e.g., Blocto, Lilico).

Click "Setup Vault" to initialize your account's BuilderToken vault. The AI server will explain what the transaction does. Confirm the transaction in your wallet and wait for it to seal.

Enter a recipient address (another testnet account or your own) and an amount. Click "Send" to transfer tokens. The AI server will preview the transaction, then explain the result after it seals.

Verify the transaction on \[Flow Testnet Explorer](https://testnet.flowscan.org/) by searching for your transaction ID. Check that events (TokensWithdrawn, TokensDeposited) were emitted correctly.

Query your balance using the Flow CLI:

\`\`\`bash
flow scripts execute cadence/scripts/get\_balance.cdc 0xYourAddress \-\-network\=testnet
\`\`\`

\#\# Conclusion

You've built a complete Flow dApp with a FungibleToken\-compatible contract, wallet authentication, and AI\-powered transaction explanations. This architecture gives users clarity and confidence when interacting with blockchain transactions, reducing friction and support burden.

From here, you can extend the contract with additional features (burn, allowances), integrate the AI server into error recovery flows, or deploy the frontend to a production host. The patterns you've learned—resource\-oriented programming, FCL integration, and AI\-guided UX—apply to any Flow application you build next.